# Dual channel oscilloscope

This example shows how a signal can be captured using a dual channel oscilloscope.

**Note:**  
Voltage range of fast analog inputs on the Red Pitaya depends on gain setting that can be set by jumpers. HV setting is for input range to ±20V, while LV sets input range to ±1V.

Create a loop back from fast outputs to the fast inputs as shown in the picture below.  
Make sure that *gain setting* is set to ±1V (LV).

![Fast loop back](img/FastIOLoopBack.png "Example of the fast loop back.")

Now start generation of the signal from the [generator notebook](gen_sync_two_channel.ipynb). Execute all cells except the last one which will disable the outputs. It is recommended that you run it when you finish with this example.

In [ ]:
from redpitaya.overlay.mercury import mercury as overlay
fpga = overlay()

Make instance of both oscilloscope channels.

In [ ]:
osc = [fpga.osc(ch, 1.0) for ch in range(fpga.MNO)]

User should specify the same timing settings for both channels:
1. Sample rate decimation is an integer in range [1, 2\*\*17] = [1, 131072].
2. Number of samples stored before and after trigger, for a forced trigger only post trigger samples are enough.

Both channels should also have the same synchronization setting.
This enables controlling reset/start/stop/trigger events for both channels by calling control functions for a single channel.

In [ ]:
for ch in osc:
    # TODO: for now bypass input filter
    ch.filter_bypass = True

    # data rate decimation 
    ch.decimation = 1

    # trigger timing [sample periods]
    N = ch.buffer_size
    ch.trigger_pre  = N//4 * 1
    ch.trigger_post = N//4 * 3
    
    # osc[0] is controlling both channels
    ch.sync_src = fpga.sync_src["osc0"]

Trigger level settings can be configured for each channel separately.

In [ ]:
# trigger level [V], edge ['neg', 'pos'] and holdoff time [sample periods]
osc[0].level   = 0.5
osc[0].edg     = 'pos'
osc[0].holdoff = 20

osc[1].level   = -0.2
osc[1].edg     = 'neg'
osc[1].holdoff = 0

Both channels should have the same trigger source which should be one of the channels.

In [ ]:
for ch in osc:
    ch.trig_src = fpga.trig_src["osc0"]

The oscilloscope should be reset and started by calling functions from the master oscilloscope module.
A loop is checking if a hardware trigger has arrived and if the desired number of data samples was already stored in the buffer. Since both channels were programmed with the same timing setting they both stop at the same time.

Data should be read from both channels.


In [ ]:
# reset and start
osc[0].reset()
osc[0].start()
# wait for data
while (osc[0].status_run()): pass
print ("triggered")

import matplotlib.pyplot as plt

# show only the part of the buffer requested by pre/post trigger timing
for ch in osc:
    data = ch.data(N)
    plt.plot(data)
    plt.show()